In [74]:
import ccxt
import os
import datetime

from dotenv import dotenv_values
from pprint import pprint
from arch import arch_model

import ccxt.async_support as ccxt # link against the asynchronous version of ccxt

from functions.BS_pricer import BS_pricer
from functions.Parameters import Option_param
from functions.Processes import Diffusion_process

import numpy as np
import scipy as scp
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial

import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
config = {
    **dotenv_values(".env.shared"),  # load shared development variables
    **dotenv_values(".env.secret"),  # load sensitive variables
    **os.environ,  # override loaded values with environment variables
}

exchange = ccxt.delta(config)
#exchange = ccxt.binance(config)

exchange.verbose = False 

exchange.check_required_credentials()

True

In [70]:
%%capture --no-display

# markets = await exchange.fetchMarkets() # Full reloading
symbol = "BTC/USDT:USDT"
markets = exchange.load_markets()
balance = exchange.fetch_balance()
currencies = exchange.fetch_currencies()
orderbook = exchange.fetch_order_book(symbol)
l2_orderbook = exchange.fetch_l2_order_book(symbol)
fetchTrades = exchange.fetch_trades(symbol)
ticker = exchange.fetch_ticker(symbol)
positions = exchange.fetch_positions()
# ledger = exchange.fetch_ledger() # not on binance

# risk = exchange.fetch_positions_risk(symbol) # not suppoted yet
# transactions = exchange.fetch_transactions() # not supported yet

In [77]:
delta_exchange_markets = await exchange.load_markets()
# Retrieve only options contract id
_ = {}
for k, v in delta_exchange_markets.items():
    if k.startswith(symbol): _.update({k:v})

btc_options = {}
for k, v in _.items():
    if v["option"]: btc_options.update({k:v})

In [ ]:
# to get IV/Greeks
delta_exchange_ticker = await exchange.fetchTickers(list(btc_options.keys()))

# to get Index price (Underlying price)
index_info = await exchange.fetch_ticker(symbol, params={'price':'index'})
index_price = index_info['last']

In [ ]:
option_contract = "BTC/USDT:USDT-221130:15900:C"
expiry_hour = 130000
contract = delta_exchange_ticker[option_contract]
pprint(contract)
a = datetime.datetime.now()
#b = datetime.datetime.strptime(contract["expiryDatetime"], "%Y-%m-%dT%H:%M:%SZ")
#seconds_until_expiration = b-a
#seconds_until_expiration

In [112]:
import re
# parse option name to isolate them
option_contract = "BTC/USDT:USDT-221130:15900:C"
def parse_option_spectification(option_contract):
    symbol, settlement_currency, expiration_date, strike, payoff = re.split(':|-', option_contract)

    payoff = "call" if payoff == "C" else "put"
    strike = float(strike)
    expiration_date = datetime.datetime.strptime(expiration_date, "%y%m%d")

    return {
        "symbol": symbol,
        "settlement_currency": settlement_currency,
        "expiration_date": expiration_date,
        "strike": strike,
        "payoff": payoff,
    }

parse_option_spectification(option_contract)

{'symbol': 'BTC/USDT',
 'settlement_currency': 'USDT',
 'expiration_date': datetime.datetime(2022, 11, 30, 0, 0),
 'strike': 15900.0,
 'payoff': 'put'}

In [ ]:
"""
    Retrieve
    IV
    index price (Underlying price)
    days until expiration
    strike price
    interest rate
    dividend yield (0)
"""
greeks = contract["info"]["greeks"]
ask_iv = float(contract["info"]["quotes"]['ask_iv'])
ask_size = float(contract["info"]["quotes"]['ask_size'])
best_ask = float(contract["info"]["quotes"]['best_ask'])
best_bid = float(contract["info"]["quotes"]['best_bid'])
bid_iv = float(contract["info"]["quotes"]['bid_iv'])
bid_size = float(contract["info"]["quotes"]['bid_size'])
mark_iv = float(contract["info"]["quotes"]['mark_iv'])
strike_price = float(contract["info"]["strike_price"])
spot_price = float(contract["info"]["spot_price"])

In [ ]:
S0=spot_price
K=strike_price
T=1/(365*24)
r=0.1
sigma=ask_iv

In [ ]:
def binomial_price(S0, K, T, r, sigma, N=15000, payoff="call"):
    # N number of periods or number of time steps  

    dT = float(T) / N                             # Delta t
    u = np.exp(sigma * np.sqrt(dT))                 # up factor
    d = 1.0 / u                                   # down factor 

    V = np.zeros(N+1)                             # initialize the price vector
    S_T = np.array( [(S0 * u**j * d**(N - j)) for j in range(N + 1)] )  # price S_T at time T

    a = np.exp(r * dT)    # risk free compounded return
    p = (a - d)/ (u - d)  # risk neutral up probability
    q = 1.0 - p           # risk neutral down probability   

    if payoff =="call":
        V[:] = np.maximum(S_T-K, 0.0)
    else:
        V[:] = np.maximum(K-S_T, 0.0)

    for i in range(N-1, -1, -1):
        V[:-1] = np.exp(-r*dT) * (p * V[1:] + q * V[:-1])    # the price vector is overwritten at each step
            
    return V[0]


In [ ]:
call_price = BS_pricer.BlackScholes("call", S0=S0, K=K, T=T ,r=r, sigma=sigma)
print(call_price) # our price
print(binomial_price(S0,K,T,r,sigma)) # binomial price
print(best_ask) # market price

In [ ]:
import ccxt
import pandas as pd

exch = 'binance' # initial exchange
t_frame = '1d' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbol = 'BTC/USDT' # initial symbol
#exchange_list = ['binance','bitfinex','kraken','poloniex','gateio','kucoin','okex','okex3','delta']
exchange_list = ['binance','delta', 'kraken', 'poloniex']
 
# Get our Exchange
try:
    exchange = getattr(ccxt, exch)()
except AttributeError:
    print('-'*36,' ERROR ','-'*35)
    print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
    print('-'*80)
    quit()
 
# Check if fetching of OHLC Data is supported
if exchange.has["fetchOHLCV"] != True:
    print('-'*36,' ERROR ','-'*35)
    print('{} does not support fetching OHLC data. Please use another  exchange'.format(exch))
    print('-'*80)
    quit()
 
# Check requested timeframe is available. If not return a helpful error.
if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
    print('-'*36,' ERROR ','-'*35)
    print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
    print('Available timeframes are:')
    for key in exchange.timeframes.keys():
        print('  - ' + key)
    print('-'*80)
    quit()
 
# Check if the symbol is available on the Exchange
exchange.load_markets()
if symbol not in exchange.symbols:
    print('-'*36,' ERROR ','-'*35)
    print('The requested symbol ({}) is not available from {}\n'.format(symbol,exch))
    print('Available symbols are:')
    for key in exchange.symbols:
        print('  - ' + key)
    print('-'*80)
    quit()
 
 
# Get data
data = exchange.fetch_ohlcv(symbol, t_frame)
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
df = pd.DataFrame(data, columns=header).set_index('Timestamp')
df['symbol'] = symbol
syms = [symbol]
filename = '{}.csv'.format(t_frame)

for exch in exchange_list:
    try:
        exchange = getattr(ccxt, exch)()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    for coin in exchange.symbols:
        if coin in syms or coin[-3:] != 'BTC':
            continue
        else:
            try:
                data = exchange.fetch_ohlcv(coin, t_frame)
            except:
                continue
            data_df = pd.DataFrame(data, columns=header).set_index('Timestamp')
            data_df['symbol'] = coin
            # df = df.append(data_df)
            df = pd.concat([df, data_df])
            syms.append(coin)
df.index = df.index/1000 #Timestamp is 1000 times bigger than it should be in this case
df['Date'] = pd.to_datetime(df.index,unit='s')
df.to_csv(filename)

In [92]:
import ccxt
import pandas as pd
from pprint import pprint

header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
# since = exchange.milliseconds () - 86400000  # -1 day from now
start_date = '2021-01-01T00:00:00Z' # fetch from a certain starting datetime
t_frame = '1h' # 1-day timeframe, usually from 1-minute to 1-week depending on the exchange
symbols = [
    'BTC/USDT:USDT',
    'AAVE/USDT:USDT',
    'BNB/USDT:USDT',
    'BTC/USDT:USDT',
    'CRV/USDT:USDT',
    'SNX/USDT:USDT',
    'LINK/USDT:USDT',
    'ETH/USDT:USDT',
]
exchange_list = ['delta']
data_df = pd.DataFrame(None, columns=header).set_index('Timestamp')

for exch in exchange_list:
    print(exch)
    try:
        exchange = getattr(ccxt, exch)()
    except AttributeError:
        print('-'*36,' ERROR ','-'*35)
        print('Exchange "{}" not found. Please check the exchange is supported.'.format(exch))
        print('-'*80)
        quit()
    if exchange.has["fetchOHLCV"] != True:
        print('-'*36,' ERROR ','-'*35)
        print('{} does not support fetching OHLC data. Please use another exchange'.format(exch))
        print('-'*80)
        quit()
    if (not hasattr(exchange, 'timeframes')) or (t_frame not in exchange.timeframes):
        print('-'*36,' ERROR ','-'*35)
        print('The requested timeframe ({}) is not available from {}\n'.format(t_frame,exch))
        print('Available timeframes are:')
        for key in exchange.timeframes.keys():
            print('  - ' + key)
        print('-'*80)
        quit()
    exchange.load_markets()
    for coin in symbols:
        print(coin)
        since = exchange.parse8601(start_date) # fetch from a certain starting datetime
        all_ohlcv = []
        while since < exchange.milliseconds():
            coin_ohlcv = exchange.fetchOHLCV(coin, t_frame, since)
            if not coin_ohlcv: break
            since = coin_ohlcv[-1][0] + 1 # query the last and add 1 to the number of milliseconds
            all_ohlcv += coin_ohlcv
        df = pd.DataFrame(all_ohlcv, columns=header).set_index('Timestamp')
        df['Symbol'] = coin
        df['Exchange'] = exch
        data_df = pd.concat([data_df, df])

data_df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
data_df['Date'] = pd.to_datetime(data_df.index, unit='s')
data_df.to_csv(f"./data/{start_date}_{t_frame}")

delta
BTC/USDT:USDT
AAVE/USDT:USDT
BNB/USDT:USDT
BTC/USDT:USDT
CRV/USDT:USDT
SNX/USDT:USDT
LINK/USDT:USDT
ETH/USDT:USDT


In [93]:
df = pd.read_csv(f"./data/{start_date}_{t_frame}", index_col="Date")
df

,Timestamp,Open,High,Low,Close,Volume,Symbol,Exchange
Date,,,,,,,,
2021-01-01 01:00:00,1.609463e+09,29000.50,29446.00,28970.00,29414.50,30051.0,BTC/USDT:USDT,delta
2021-01-01 02:00:00,1.609466e+09,29430.00,29453.50,29162.00,29221.50,20332.0,BTC/USDT:USDT,delta
2021-01-01 03:00:00,1.609470e+09,29228.00,29346.50,29202.00,29300.00,16980.0,BTC/USDT:USDT,delta
2021-01-01 04:00:00,1.609474e+09,29333.00,29387.50,29057.50,29228.00,16629.0,BTC/USDT:USDT,delta
2021-01-01 05:00:00,1.609477e+09,29223.00,29228.00,29112.00,29205.50,16948.0,BTC/USDT:USDT,delta
...,...,...,...,...,...,...,...,...
2022-11-30 18:00:00,1.669831e+09,1265.50,1284.15,1258.70,1278.80,63280.0,ETH/USDT:USDT,delta
2022-11-30 19:00:00,1.669835e+09,1279.90,1298.40,1279.35,1285.55,84002.0,ETH/USDT:USDT,delta
2022-11-30 20:00:00,1.669838e+09,1286.90,1294.25,1284.10,1294.25,16990.0,ETH/USDT:USDT,delta


In [96]:
returns = 100 * df[df["Symbol"] == "BTC/USDT:USDT"]["Close"].pct_change().dropna()
returns

Date
2021-01-01 02:00:00   -0.656139
2021-01-01 03:00:00    0.268638
2021-01-01 04:00:00   -0.245734
2021-01-01 05:00:00   -0.076981
2021-01-01 06:00:00   -0.082176
                         ...   
2022-11-30 18:00:00    0.795448
2022-11-30 19:00:00    0.747791
2022-11-30 20:00:00    0.278707
2022-11-30 21:00:00    0.040958
2022-11-30 22:00:00   -0.081883
Name: Close, Length: 33547, dtype: float64

In [97]:
call_price = BS_pricer.BlackScholes("call", S0=S0, K=K, T=T ,r=r, sigma=annualized_volatility_forecast)
print(call_price) # our price

NameError: name 'S0' is not defined

In [ ]:
base = datetime.datetime.today()

In [ ]:
s = exchange.parse8601(base)

In [ ]:
exchange.milliseconds()

In [50]:
df['symbol'] = symbol
df['exchange'] = exch
df.index /= 1000 #Timestamp is 1000 times bigger than it should be in this case
df['Date'] = pd.to_datetime(df.index, unit='s')
df.to_csv(f"./data/{exch}_{t_frame}")

In [51]:
len(all_ohlcv)

16771